In [1]:
import pandas as pd
import numpy as np
import os
import gensim
import sklearn
from sklearn import ensemble,svm,neural_network,discriminant_analysis
from sklearn.metrics import roc_curve,auc,precision_recall_curve
from matplotlib import pyplot as plt

with open("sent_corpus.csv", "r") as sent_file:
    lines = sent_file.read().split("\n")

rows = [line.split(",") for line in lines if line]
rows = [row[:3] + [",".join(row[3:])] for row in rows]
# remove document start character
rows[0][0] = rows[0][0][1:]
sentDf_cols = ['ItemID', 'Sentiment', 'SentimentSource', 'SentimentText']
sentDf = pd.DataFrame(rows[1:],columns=sentDf_cols)
# print(sentDf.columns.values)
sentDf[["ItemID","Sentiment"]] = sentDf[["ItemID","Sentiment"]].astype(int)
sentDf["SentimentText"] = sentDf["SentimentText"].apply(lambda text: text.split())
## -*- coding: utf-8 -*-
curdir = os.getcwd()
tweet_path = curdir + "/crawler_v1/code/tweettext.txt"
print(tweet_path)

tweets = []
tweet_file = open(tweet_path, "rb")
for line in tweet_file:
#    print(line)
#    line = line_pre.decode('utf-8')
    tweet = ""
    if(line[:2] == "RT"):
        char_loc = 0
        for char in line:
            char_loc += 1
            if(char==":"):
                break

        tweet = line[-(len(line)-char_loc):]
    else:
        tweet = line
    tweets.append(tweet)

#[tweet.decode('utf-8') for tweet in tweets]
# print(tweets)
# print("done")


/home/zeyuanxu/Desktop/testdir/crawler_v1/code/tweettext.txt


In [2]:
sentDf

,ItemID,Sentiment,SentimentSource,SentimentText
0,1,0,Sentiment140,"[is, so, sad, for, my, APL, friend.............]"
1,2,0,Sentiment140,"[I, missed, the, New, Moon, trailer...]"
2,3,1,Sentiment140,"[omg, its, already, 7:30, :O]"
3,4,0,Sentiment140,"[.., Omgaga., Im, sooo, im, gunna, CRy., I've,..."
4,5,0,Sentiment140,"[i, think, mi, bf, is, cheating, on, me!!!, T_T]"
5,6,0,Sentiment140,"[or, i, just, worry, too, much?]"
6,7,1,Sentiment140,"[Juuuuuuuuuuuuuuuuussssst, Chillin!!]"
7,8,0,Sentiment140,"[Sunny, Again, Work, Tomorrow, :-|, TV, Tonight]"
8,9,1,Sentiment140,"[handed, in, my, uniform, today, ., i, miss, y..."
9,10,1,Sentiment140,"[hmmmm...., i, wonder, how, she, my, number, @-)]"


In [3]:
size_list = [100, 110, 120, 130, 140, 150]

In [4]:
# convertion i = (x - 100)/10 

In [5]:
Res = np.zeros((10,6))

In [16]:
from sklearn.model_selection import KFold

In [ ]:
# test 
# currently freezes 
size_s = 100
window_s = 5 
total_f1 = 0 
kf = KFold(n_splits=10, shuffle=True)
for train_indeces, test_indeces in kf.split(sentdat):
    train_X = sentDf["SentimentText"].iloc[train_indeces]
    train_Y = sentDf["Sentiment"].iloc[train_indeces]
    test_X = sentDf["SentimentText"].iloc[test_indeces]
    test_Y = sentDf["Sentiment"].iloc[test_indeces]
    # Ns = int(1e5)
    # training set word2vec 
    w2vM_train = gensim.models.Word2Vec(train_X, size=size_s, window=window_s)
    tvecs_train = np.array([np.array([w2vM_train[t] if t in w2vM_train
                                else np.zeros((size_s,))                               
                            for t in twt]).mean(axis=0)
                     for twt in train_X])
    # test set word2vec 
    w2vM_test = gensim.models.Word2Vec(test_X, size=size_s, window=window_s)
    tvecs_test = np.array([np.array([w2vM_test[t] if t in w2vM_test
                                else np.zeros((size_s,))                               
                            for t in twt]).mean(axis=0)
                     for twt in train_X])
   
    # training
    mlp = sklearn.neural_network.MLPClassifier()
    mlp.fit(tvecs_train,train_Y)
    yhat = mlp.predict(tvecs_test)
    accu = np.mean(yhat == test_Y)
    prec = np.mean(test_Y[yhat == 1])
    recl = np.mean(yhat[test_Y == 1])
    f1 = 2 * prec * recl / (prec + recl)
    print(f1)
    total_f1 += f1 
avgf1 = total_f1 / 10 
print(avgf1)

In [28]:
%%time 
sentdat = sentDf["SentimentText"]
for window_s in range(1, 11):
    for size_s in size_list:
        total_f1 = 0 
        kf = KFold(n_splits=10, shuffle=True)
        for train_indeces, test_indeces in kf.split(sentdat):
            train_X = sentDf["SentimentText"].iloc[train_indeces]
            train_Y = sentDf["Sentiment"].iloc[train_indeces]
            test_X = sentDf["SentimentText"].iloc[test_indeces]
            test_Y = sentDf["Sentiment"].iloc[test_indeces]
            Ns = int(1e5)
            # training set word2vec 
            w2vM_train = gensim.models.Word2Vec(train_X, size=size_s, window=window_s)
            tvecs_train = np.array([np.array([w2vM_train[t] if t in w2vM_train
                                else np.zeros((size_s,))                               
                            for t in twt]).mean(axis=0)
                     for twt in sentDf["SentimentText"][:Ns]])
            # test set word2vec 
            w2vM_test = gensim.models.Word2Vec(test_X, size=size_s, window=window_s)
            tvecs_test = np.array([np.array([w2vM_test[t] if t in w2vM_test
                                else np.zeros((size_s,))                               
                            for t in twt]).mean(axis=0)
                     for twt in sentDf["SentimentText"][:Ns]])
   
            # training
            mlp = sklearn.neural_network.MLPClassifier()
            mlp.fit(tvecs_train,train_Y)
            yhat = mlp.predict(tvecs_test)
            accu = np.mean(yhat == test_Y)
            prec = np.mean(test_Y[yhat == 1])
            recl = np.mean(yhat[test_Y == 1])
            f1 = 2 * prec * recl / (prec + recl)
            print(f1)
            total_f1 += f1 
        avgf1 = total_f1 / 10 
    # store the f1 score in the grid as the metrics
    Res[window_s-1, (size_s - 100)/10 ] = avgf1 

KeyboardInterrupt: 

In [26]:
Res

array([[ 0.81004103,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ]])